# Part 1: Option Valuation

## Ex. 1: Binomial tree program to approximate the price of the option. 
How does your estimate compare to the analytical value? <br>
Experiment for different values of the volatility

In [20]:
import math
from scipy.stats import norm

stock_price = 100.
strike_price = 99.0
interest_rate = 0.06
volatility = 0.2
steps = 50
T = 1 # maturity time in years

u = math.exp(volatility * math.sqrt(1/steps))
d = math.exp(-volatility * math.sqrt(1/steps))
a = math.exp(interest_rate * 1/steps)
p = (a - d) / (u - d)


''' The bionomial tree. ''' 
def binomial_tree(steps, start):
    ''' Binomial tree to approximate the price of the option. ''' 
    # Result includes stock price [0], option price [1], delta [2]
    results = [[[start, 0, 0]]]
    
    for i in range(steps): 
        result = []
        for j in range(len(results[-1])):
            # calculate the stock price [0]
            if j == 0:
                result.append([results[-1][j][0] * d, 0, 0])
            result.append([results[-1][j][0] * u, 0, 0])
        results.append(result)  
   
    return results


''' Approximate value with the european price call. ''' 
def call_option(S, K):
    ''' Calculates the value of a call option at its expiration date. '''
    return max(0, S - K)

def risk_neutral_valuation(r, dt, fi, fj, p): 
    ''' Risk-neutral valuation of the option. '''
    return math.exp(-r * dt) * (p * fi + (1 - p)*fj)

def price_EU_call(treelist, strike_price, p):
    ''' The price of the European call. '''
    lentree = len(treelist)
    
    # calculate the option prices [1]
    for el in treelist[-1]:
        el[1] = call_option(el[0], strike_price)
        
    # calculate risk neutral valuation: back in time [1]
    for i in range(len(treelist) - 2, -1, -1): # i is time
        for j in range(len(treelist[i])): # j is ????
            treelist[i][j][1] = risk_neutral_valuation(interest_rate, 1/steps, treelist[i+1][j+1][1], treelist[i+1][j][1], p)
   
    return treelist


'''Analytical value'''
def black_scholes(stock_price, strike_price, interest_rate, volatility, T):
    d1 = (math.log(stock_price / strike_price) + (interest_rate + (volatility ** 2) / 2) * T) / (volatility * math.sqrt(T))
    #d2 = d1 - volatility* math.sqrt(T)
    d2 = (math.log(stock_price / strike_price) + (interest_rate - (volatility ** 2) / 2) * T) / (volatility * math.sqrt(T))
   
    call = stock_price * norm.cdf(d1) - strike_price * math.exp(- interest_rate * T) * norm.cdf(d2)
    put = strike_price * math.exp(-interest_rate * T ) * norm.cdf(-d2) - stock_price * norm.cdf(-d1)

    return call, put

In [21]:
''' The estimate compared to the analytical value '''
treelist = binomial_tree(steps, stock_price)
filled_tree = price_EU_call(treelist, strike_price, p)

#The estimate 
estimated_price = filled_tree[0][0][1]
print('The estimated price of the option:', estimated_price)

#The analytical value
analytical_call, analytical_put = black_scholes(stock_price, strike_price, interest_rate, volatility, T)
print('The analytical price of the option:', analytical_call)


The estimated price of the option: 11.546434850755034
The analytical price of the option: 11.544280227051097


In [134]:
''' Different values of volatility '''
import matplotlib.pyplot as plt
import numpy as np

plt.figure()

volatility = np.linspace(0,1,100)

for i in volatility: 
    treelist = binomial_tree(steps, stock_price)
    filled_tree = price_EU_call(treelist, strike_price)

    # Estimated  and analytical price
    estimated_price = filled_tree[0][0][1]  
    
plt.plot(len(estimated_prices), estimated_price, label ='Estimated price')
plt.plot(len(analytical_prices, analytical_price, label ='Analytical price')

plt.title('Approximation of the option price')
plt.xlabel('Volatility (sigma)')
plt.ylabel('Option price')
plt.legend(loc = 'best')
plt.show()

SyntaxError: invalid syntax (<ipython-input-134-3b2fca75fd39>, line 19)

## Ex. 2: Convergence of the method
What is the computational complexity of this algorithm? <br>
Done

## Ex. 3: Hedge parameter
Compare with the analytical values. 
Experiment for different values of the volatility.

In [26]:
''' The estimation of delta. '''
def estimate_delta(treelist):
    lentree = len(treelist)
    
    for i in range(len(treelist) - 2, -1, -1): # i is time
        for j in range(len(treelist[i])): # j is ????
            treelist[i][j][2] = (treelist[i+1][j+1][1] - treelist[i+1][j][1]) / (treelist[i+1][j+1][0] - treelist[i+1][j][0])
 
    return treelist
            
#SAME CALCULATIOIN, DIFFERENT METHOD TO ESTIMATE THE DELTA
def calc_delta(treelist):
    lentree = len(treelist)
    for j in range(lentree - 1):
        for k in range(len(treelist[-(j+2)])):
            treelist[-(j+2)][k][2] = (treelist[-(j+1)][k][1] - treelist[-(j+1)][k+1][1]) / (treelist[-(j+1)][k][0] - treelist[-(j+1)][k+1][0])
#             if j == 0 and k ==0:
#                 print(treelist[-(j+1)][k][1], treelist[-(j+1)][k+1][1], treelist[-(j+2)][k][1])
    return treelist


''' The analytical delta. '''
def analytical_delta(treelist):
    return None 

In [28]:
''' The estimate compared to the analytical value. '''
# The estimate 
estimated_deltas = estimate_delta(filled_tree)
print(estimated_deltas[0])

b = calc_delta(filled_tree)
print(b[0])

# The analytical


[[100.0, 11.546434850755034, 0.6725569646047738]]
[[100.0, 11.546434850755034, 0.6725569646047738]]


## Ex. 4: American option 
What is the value of the American put and call for the corresponding parameters? <br>
Experiment for different values of the volatility.

In [37]:
''' American call option. ''' 
def price_US_call(treelist, strike_price):
    lentree = len(treelist)
    for el in treelist[-1]:
        el[1] = call_option(el[0], strike_price)
        
    for i in range(lentree - 1):
        for j in range(len(treelist[-(i+2)])):
            keeping_value = risk_neutral_valuation(interest_rate, 1/steps, treelist[-(i+1)][j+1][1], treelist[-(i+1)][j][1], p)
            use_now_value = call_option(treelist[-(i+2)][j][0], strike_price)
            
            treelist[-(i+2)][j][1] = max(keeping_value, use_now_value)

    return treelist


''' American put option. ''' 
def put_option(): 
    return 

def price_US_put(treelist, strike_price):
    lentree = len(treelist)
    for el in treelist[-1]:
        el[1] = put_option(el[0], strike_price)
        
    for i in range(lentree - 1):
        for j in range(len(treelist[-(i+2)])):
            keeping_value = risk_neutral_valuation(interest_rate, 1/steps, treelist[-(i+1)][j+1][1], treelist[-(i+1)][j][1], p)
            use_now_value = put_option(treelist[-(i+2)][j][0], strike_price)
            
            treelist[-(i+2)][j][1] = max(keeping_value, use_now_value)

    return treelist
    

'American put option. '

In [38]:


treelist = binomial_tree(steps, stock_price)
print(price_US_call(treelist, strike_price))

[[[100.0, 11.546434850755034, 0]], [[97.21119840328973, 9.603915673337521, 0], [102.86880693018583, 13.408979691108847, 0]], [[94.50017095003759, 7.867993922459337, 0], [100.0, 11.266700206672372, 0], [105.81991439239847, 15.464637347516563, 0]], [[91.864748673689, 6.33883905009713, 0], [97.21119840328971, 9.331340500413575, 0], [102.86880693018583, 13.121826442471455, 0], [108.85568343000432, 17.714587466223392, 0]], [[89.30282309586327, 5.013389178999797, 0], [94.50017095003759, 7.606123858779705, 0], [99.99999999999999, 10.9833727046514, 0], [105.81991439239847, 15.17319836638081, 0], [111.97854282014544, 20.15743808873055, 0]], [[86.81234453945848, 3.8850593201676125, 0], [91.864748673689, 6.0913104121102375, 0], [97.21119840328971, 9.055306977004804, 0], [102.86880693018581, 12.830944842779047, 0], [108.85568343000432, 17.421941179744802, 0], [115.19099101689088, 22.789415420177466, 0]], [[84.39132048880043, 2.9436699706749256, 0], [89.30282309586326, 4.783696422695615, 0], [94.50

# Part 2: Hedging Simulations

In [ ]:
# T = 1

stock_price = 100.0
strike_price = 99.0
interest_rate = 1.06
volatility = 0.20

def dynamics_stock_price(r, S, dt, sigma, dZ):
    return r*S*dt + sigma * S * dZ

